# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [17]:
quantity_df = pd.DataFrame(data.groupby(['CustomerID','ProductName']).Quantity.sum()).reset_index()
quantity_df

# customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
# customer_products

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
...,...,...,...
63623,98200,Vol Au Vents,50
63624,98200,Wasabi Powder,25
63625,98200,Wine - Fume Blanc Fetzer,25
63626,98200,Wine - Hardys Bankside Shiraz,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [120]:
prod_cst_mtx = quantity_df.pivot_table(values = 'Quantity',
                            index = 'ProductName', 
                            columns = 'CustomerID').fillna(0)



In [123]:
prod_cst_mtx.reset_index(level = 'ProductName')

CustomerID,ProductName,33,200,264,356,412,464,477,639,649,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
0,Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
3,Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
4,Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
448,Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
449,"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
450,Yogurt - French Vanilla,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0


In [ ]:


#pd.DataFrame(data = prod_cst_mtx, columns = ['ProductName'])

In [13]:
cst_prod_mtx = prod_cst_mtx.T
cst_prod_mtx.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [124]:
prod_cst_mtx = prod_cst_mtx.div(prod_cst_mtx.sum(axis=1), axis=0)
cst_prod_mtx = cst_prod_mtx.div(cst_prod_mtx.sum(axis=1), axis=0)

In [125]:
prod_cst_mtx.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000521,0.0,0.0,...,0.000000,0.013034,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.011221,0.011221,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
Appetizer - Mushroom Tart,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000532,0.0,0.0,...,0.013298,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.013298,0.0
Appetizer - Sausage Rolls,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.014384,0.014384,0.014384,0.0,0.014384,0.0
Apricots - Dried,0.000491,0.0,0.0,0.0,0.000491,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.012285,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [126]:
def distancia(summary, metric='euclidean'):
    from scipy.spatial.distance import squareform, pdist
    pairwise = pd.DataFrame(1/(1+squareform(pdist(summary, metric))),
        columns = summary.index,
        index = summary.index
    )
    return pairwise

In [127]:
cst_sim_matrix = distancia(cst_prod_mtx)
cst_sim_matrix
#cst_sim_matrix.reset_index

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.846485,0.854046,0.860153,0.851289,0.856629,0.855545,0.856079,0.851250,0.853632,0.849471,...,0.857702,0.854124,0.850987,0.851150,0.862796,1.000000,0.862919,0.855458,0.858954,0.851527
98069,0.852848,0.868990,0.859773,0.851219,0.855052,0.852574,0.857118,0.851869,0.855343,0.855701,...,0.859570,0.855797,0.855409,0.847469,0.861720,0.862919,1.000000,0.859010,0.864271,0.858525
98159,0.847637,0.857312,0.854246,0.852260,0.856608,0.852782,0.854378,0.851396,0.855557,0.851331,...,0.859624,0.858006,0.853659,0.845236,0.855405,0.855458,0.859010,1.000000,0.861266,0.854102


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [218]:
similar_cust = list(cst_sim_matrix[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [223]:
kkn_cust = quantity_df[quantity_df['CustomerID'].isin(similar_cust)].sort_values('Quantity', ascending=False)
kkn_cust

,CustomerID,ProductName,Quantity
50798,79458,Wine - Sogrape Mateus Rose,42
50797,79458,Wine - Redchard Merritt,42
50739,79458,Cassis,42
50789,79458,"Thyme - Lemon, Fresh",42
39502,60862,Cake - Box Window 10x10x2.5,32
...,...,...,...
4473,6001,Soup - Campbells Tomato Ravioli,2
4474,6001,Soupcontfoam16oz 116con,2
4475,6001,Soupfoamcont12oz 112con,2
4477,6001,Squid - Tubes / Tenticles 10/20,2


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [245]:
kkn_cust_df = pd.DataFrame(kkn_cust.groupby(['ProductName']).Quantity.sum().sort_values(ascending = False))
kkn_cust_df 

,Quantity
ProductName,
Ocean Spray - Kiwi Strawberry,47
Sausage - Liver,45
Tomatoes Tear Drop,45
Wine - White Cab Sauv.on,41
Fond - Neutral,34
...,...
Sauce - Hollandaise,1
Scallops - 10/20,1
Scallops 60/80 Iqf,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [230]:
merged = pd.merge(kkn_cust_df, cst_prod_mtx.iloc[33], on = 'ProductName' ).reset_index()
merged.columns = ['ProductName', 'Quantity', 'Purchased'] 

In [253]:
reco_33 = merged[merged['Purchased'] == 0].head()
reco_33

,ProductName,Quantity,Purchased
0,Jolt Cola - Electric Blue,89,0.0
1,Lettuce - Treviso,84,0.0
2,Watercress,83,0.0
3,Cookie Dough - Double,66,0.0
4,Cheese - Camembert,61,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [277]:
recommendations = {}
customers = quantity_df['CustomerID'].unique()

In [278]:
for customer in customers:
    similar_cust = list(cst_sim_matrix[customer].sort_values(ascending=False)[1:].head().index)
    kkn_cust = quantity_df[quantity_df['CustomerID'].isin(similar_cust)].sort_values('Quantity', ascending=False)
    kkn_cust_df = pd.DataFrame(kkn_cust.groupby(['ProductName']).Quantity.sum().sort_values(ascending = False))

    merged = pd.merge(kkn_cust_df, cst_prod_mtx.iloc[customer], on = 'ProductName' ).reset_index()
    merged.columns = ['ProductName', 'Quantity', 'Purchased'] 
    reco = merged[merged['Purchased'] == 0].head()
    
    recommendations[customer] = list(reco['ProductName'])
    print(recommendations)

{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda'], 264: ['Eggplant - Asian', 'Scallops - 10/20', 'Wine - Ej Gallo Sierra Valley', 'Scallop - St. Jaques', 'Turnip - White, Organic']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie 

IndexError: single positional indexer is out-of-bounds

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [279]:
recommendations_euclidean = pd.DataFrame(recommendations)
recommendations_euclidean

,33,200,264,356,412,464,477,639,649,669,694,756,883,891
0,Wine - Redchard Merritt,Sauce - Demi Glace,Eggplant - Asian,Scallops - 10/20,"Mushrooms - Black, Dried",Vaccum Bag 10x13,Lettuce - Spring Mix,Ketchup - Tomato,Soup - Campbells Tomato Ravioli,Bread - Raisin Walnut Oval,Pecan Raisin - Tarts,Muffin - Carrot Individual Wrap,Jolt Cola - Electric Blue,Jolt Cola - Electric Blue
1,Cassis,Olive - Spread Tapenade,Scallops - 10/20,Tea - Herbal Sweet Dreams,Cake - Box Window 10x10x2.5,Coffee - Dark Roast,Chocolate - Feathers,Ice Cream Bar - Hageen Daz To,Sauce - Demi Glace,Juice - V8 Splash,"Soup - Campbells, Beef Barley",Beef - Top Sirloin,Pears - Bosc,Lettuce - Treviso
2,Wine - Crozes Hermitage E.,Tahini Paste,Wine - Ej Gallo Sierra Valley,Curry Paste - Madras,Bread - Raisin Walnut Oval,Lambcasing,Bacardi Breezer - Tropical,"Crab - Dungeness, Whole",Beef - Montreal Smoked Brisket,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky,Longos - Grilled Salmon With Bbq,Scallops - 10/20,Watercress
3,Bread - Calabrese Baguette,Cookie Chocolate Chip With,Scallop - St. Jaques,Beef - Ground Medium,Beef - Montreal Smoked Brisket,Bay Leaf,Carbonated Water - Blackcherry,Juice - Apple Cider,Rosemary - Dry,Wine - Ruffino Chianti,"Mushrooms - Black, Dried",Steam Pan - Half Size Deep,Cumin - Whole,Cookie Dough - Double
4,Wine - Sogrape Mateus Rose,Pop Shoppe Cream Soda,"Turnip - White, Organic",Tea - English Breakfast,"Mushroom - Trumpet, Dry","Oregano - Dry, Rubbed",Puree - Mocha,Soupfoamcont12oz 112con,Bay Leaf,Knife Plastic - White,Mangoes,"Pepsi - Diet, 355 Ml","Water - Mineral, Natural",Cheese - Camembert


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [282]:
cst_sim_matrix_jaccard = distancia(cst_prod_mtx, metric = 'cityblock')
recommendations = {}
customers = quantity_df['CustomerID'].unique()

for customer in customers:
    similar_cust = list(cst_sim_matrix[customer].sort_values(ascending=False)[1:].head().index)
    kkn_cust = quantity_df[quantity_df['CustomerID'].isin(similar_cust)].sort_values('Quantity', ascending=False)
    kkn_cust_df = pd.DataFrame(kkn_cust.groupby(['ProductName']).Quantity.sum().sort_values(ascending = False))

    merged = pd.merge(kkn_cust_df, cst_prod_mtx.iloc[customer], on = 'ProductName' ).reset_index()
    merged.columns = ['ProductName', 'Quantity', 'Purchased'] 
    reco = merged[merged['Purchased'] == 0].head()
    
    recommendations[customer] = list(reco['ProductName'])
    print(recommendations)

{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda'], 264: ['Eggplant - Asian', 'Scallops - 10/20', 'Wine - Ej Gallo Sierra Valley', 'Scallop - St. Jaques', 'Turnip - White, Organic']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie 

IndexError: single positional indexer is out-of-bounds

In [283]:
recommendations_cityblock = pd.DataFrame(recommendations)
recommendations_cityblock

,33,200,264,356,412,464,477,639,649,669,694,756,883,891
0,Wine - Redchard Merritt,Sauce - Demi Glace,Eggplant - Asian,Scallops - 10/20,"Mushrooms - Black, Dried",Vaccum Bag 10x13,Lettuce - Spring Mix,Ketchup - Tomato,Soup - Campbells Tomato Ravioli,Bread - Raisin Walnut Oval,Pecan Raisin - Tarts,Muffin - Carrot Individual Wrap,Jolt Cola - Electric Blue,Jolt Cola - Electric Blue
1,Cassis,Olive - Spread Tapenade,Scallops - 10/20,Tea - Herbal Sweet Dreams,Cake - Box Window 10x10x2.5,Coffee - Dark Roast,Chocolate - Feathers,Ice Cream Bar - Hageen Daz To,Sauce - Demi Glace,Juice - V8 Splash,"Soup - Campbells, Beef Barley",Beef - Top Sirloin,Pears - Bosc,Lettuce - Treviso
2,Wine - Crozes Hermitage E.,Tahini Paste,Wine - Ej Gallo Sierra Valley,Curry Paste - Madras,Bread - Raisin Walnut Oval,Lambcasing,Bacardi Breezer - Tropical,"Crab - Dungeness, Whole",Beef - Montreal Smoked Brisket,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky,Longos - Grilled Salmon With Bbq,Scallops - 10/20,Watercress
3,Bread - Calabrese Baguette,Cookie Chocolate Chip With,Scallop - St. Jaques,Beef - Ground Medium,Beef - Montreal Smoked Brisket,Bay Leaf,Carbonated Water - Blackcherry,Juice - Apple Cider,Rosemary - Dry,Wine - Ruffino Chianti,"Mushrooms - Black, Dried",Steam Pan - Half Size Deep,Cumin - Whole,Cookie Dough - Double
4,Wine - Sogrape Mateus Rose,Pop Shoppe Cream Soda,"Turnip - White, Organic",Tea - English Breakfast,"Mushroom - Trumpet, Dry","Oregano - Dry, Rubbed",Puree - Mocha,Soupfoamcont12oz 112con,Bay Leaf,Knife Plastic - White,Mangoes,"Pepsi - Diet, 355 Ml","Water - Mineral, Natural",Cheese - Camembert


In [281]:
cst_sim_matrix_jaccard = distancia(cst_prod_mtx, metric = 'jaccard')
recommendations = {}
customers = quantity_df['CustomerID'].unique()

for customer in customers:
    similar_cust = list(cst_sim_matrix[customer].sort_values(ascending=False)[1:].head().index)
    kkn_cust = quantity_df[quantity_df['CustomerID'].isin(similar_cust)].sort_values('Quantity', ascending=False)
    kkn_cust_df = pd.DataFrame(kkn_cust.groupby(['ProductName']).Quantity.sum().sort_values(ascending = False))

    merged = pd.merge(kkn_cust_df, cst_prod_mtx.iloc[customer], on = 'ProductName' ).reset_index()
    merged.columns = ['ProductName', 'Quantity', 'Purchased'] 
    reco = merged[merged['Purchased'] == 0].head()
    
    recommendations[customer] = list(reco['ProductName'])
    print(recommendations)


{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie Chocolate Chip With', 'Pop Shoppe Cream Soda'], 264: ['Eggplant - Asian', 'Scallops - 10/20', 'Wine - Ej Gallo Sierra Valley', 'Scallop - St. Jaques', 'Turnip - White, Organic']}
{33: ['Wine - Redchard Merritt', 'Cassis', 'Wine - Crozes Hermitage E.', 'Bread - Calabrese Baguette', 'Wine - Sogrape Mateus Rose'], 200: ['Sauce - Demi Glace', 'Olive - Spread Tapenade', 'Tahini Paste', 'Cookie 

IndexError: single positional indexer is out-of-bounds

In [275]:
recommendations_jaccard = pd.DataFrame(recommendations)
recommendations_jaccard

,33,200,264,356,412,464,477,639,649,669,694,756,883,891
0,Wine - Redchard Merritt,Sauce - Demi Glace,Eggplant - Asian,Scallops - 10/20,"Mushrooms - Black, Dried",Vaccum Bag 10x13,Lettuce - Spring Mix,Ketchup - Tomato,Soup - Campbells Tomato Ravioli,Bread - Raisin Walnut Oval,Pecan Raisin - Tarts,Muffin - Carrot Individual Wrap,Jolt Cola - Electric Blue,Jolt Cola - Electric Blue
1,Cassis,Olive - Spread Tapenade,Scallops - 10/20,Tea - Herbal Sweet Dreams,Cake - Box Window 10x10x2.5,Coffee - Dark Roast,Chocolate - Feathers,Ice Cream Bar - Hageen Daz To,Sauce - Demi Glace,Juice - V8 Splash,"Soup - Campbells, Beef Barley",Beef - Top Sirloin,Pears - Bosc,Lettuce - Treviso
2,Wine - Crozes Hermitage E.,Tahini Paste,Wine - Ej Gallo Sierra Valley,Curry Paste - Madras,Bread - Raisin Walnut Oval,Lambcasing,Bacardi Breezer - Tropical,"Crab - Dungeness, Whole",Beef - Montreal Smoked Brisket,"Chocolate - Semi Sweet, Calets",Cinnamon Buns Sticky,Longos - Grilled Salmon With Bbq,Scallops - 10/20,Watercress
3,Bread - Calabrese Baguette,Cookie Chocolate Chip With,Scallop - St. Jaques,Beef - Ground Medium,Beef - Montreal Smoked Brisket,Bay Leaf,Carbonated Water - Blackcherry,Juice - Apple Cider,Rosemary - Dry,Wine - Ruffino Chianti,"Mushrooms - Black, Dried",Steam Pan - Half Size Deep,Cumin - Whole,Cookie Dough - Double
4,Wine - Sogrape Mateus Rose,Pop Shoppe Cream Soda,"Turnip - White, Organic",Tea - English Breakfast,"Mushroom - Trumpet, Dry","Oregano - Dry, Rubbed",Puree - Mocha,Soupfoamcont12oz 112con,Bay Leaf,Knife Plastic - White,Mangoes,"Pepsi - Diet, 355 Ml","Water - Mineral, Natural",Cheese - Camembert


In [286]:
recommendations_euclidean == recommendations_cityblock

,33,200,264,356,412,464,477,639,649,669,694,756,883,891
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [285]:
recommendations_euclidean == recommendations_jaccard

,33,200,264,356,412,464,477,639,649,669,694,756,883,891
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True
